In [71]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from datetime import datetime

pd.set_option('display.max_rows', 5000)

zhi_df2=pd.read_csv('../data/Zip_Zri_MultiFamilyResidenceRental.csv')


zri = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

#zri=zri[zri['year-month']>'2014-02']

zri=zri.fillna(0)
zri['gapsize']=zri['ZRI'].rolling(6).sum()
droplist=list(zri[(zri['gapsize']==0) & (zri['year-month']>'2014-06')]['RegionName'].drop_duplicates())
zri[(zri['gapsize']==0)].shape[0]/zri.shape[0]
len(droplist)/len(zri['RegionName'].drop_duplicates())

0.30037614185921546

In [72]:
zhi_df2.shape

(1861, 120)

In [73]:
date_mapping={i:j for i,j in enumerate(zhi_df.columns[6:])}

In [84]:
zri2 = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())]=zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())].fillna(method='ffill')

zri2=zri2.groupby('RegionName').apply(lambda group: group.interpolate())
#zri2[zri2['ZRI'].isna()]

#zri2['ZRI']=(zri2['ZRI']/zri2['ZRI'].shift(1))-1

zri2=zri2[(zri2['year-month']>'2014-01') & (zri2['year-month']<'2020-01')]

zri3=zri2[~(zri2['RegionName'].isin(droplist))]

zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')



<ipython-input-84-585f0312ac37>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [85]:
#zip codes to drop
len(droplist)

559

In [86]:
#remaining filterd ZRI data:
len(zri3['RegionName'].drop_duplicates())

1302

In [87]:
zri3.isna().sum()

RegionID       0
RegionName     0
City           0
State          0
Metro         70
CountyName     0
year-month     0
ZRI            0
dtype: int64

In [88]:
zri3

,RegionID,RegionName,City,State,Metro,CountyName,year-month,ZRI
77798,58207,1013,Chicopee,MA,Springfield,Hampden County,2014-02,931.0
79659,58207,1013,Chicopee,MA,Springfield,Hampden County,2014-03,934.0
81520,58207,1013,Chicopee,MA,Springfield,Hampden County,2014-04,929.0
83381,58207,1013,Chicopee,MA,Springfield,Hampden County,2014-05,929.0
85242,58207,1013,Chicopee,MA,Springfield,Hampden County,2014-06,929.0
...,...,...,...,...,...,...,...,...
200463,100334,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-08,1277.0
202324,100334,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-09,1271.0
204185,100334,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-10,1299.0
206046,100334,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-11,1261.5


<H1>Lag development

In [9]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [10]:
zri3=laggenerator(1, 'ZRI',zri3)
zri3=laggenerator(2, 'ZRI',zri3)
zri3=laggenerator(3, 'ZRI',zri3)
zri3=laggenerator(4, 'ZRI',zri3)
zri3=laggenerator(5, 'ZRI',zri3)
zri3=laggenerator(6, 'ZRI',zri3)

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [11]:
train, test=zri3[(zri3['year-month']<='2018-12') & (zri3['year-month']>'2014-05')], zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')]

train=train.drop('year-month',axis=1)

Y_train, Y_test=train['ZRI'], test[['ZRI']]
X_train, X_test=train.drop('ZRI',axis=1), test.drop('ZRI',axis=1)



In [12]:
labelencoder = LabelEncoder()

X_train['CountyNameEnc'] = labelencoder.fit_transform(X_train['CountyName'])

X_train=X_train.drop(['CountyName','RegionName','City','State','Metro'],axis=1)


In [13]:
cat=CatBoostRegressor()

In [14]:
cat.fit(X_train, Y_train)

Learning rate set to 0.083065
0:	learn: 0.0092220	total: 63.5ms	remaining: 1m 3s
1:	learn: 0.0089780	total: 69ms	remaining: 34.4s
2:	learn: 0.0087797	total: 73.1ms	remaining: 24.3s
3:	learn: 0.0085878	total: 78.1ms	remaining: 19.5s
4:	learn: 0.0084261	total: 83.1ms	remaining: 16.5s
5:	learn: 0.0082804	total: 87.5ms	remaining: 14.5s
6:	learn: 0.0081503	total: 92.4ms	remaining: 13.1s
7:	learn: 0.0080406	total: 97.4ms	remaining: 12.1s
8:	learn: 0.0079453	total: 102ms	remaining: 11.2s
9:	learn: 0.0078601	total: 106ms	remaining: 10.5s
10:	learn: 0.0077872	total: 111ms	remaining: 10s
11:	learn: 0.0077237	total: 116ms	remaining: 9.54s
12:	learn: 0.0076719	total: 120ms	remaining: 9.14s
13:	learn: 0.0076229	total: 125ms	remaining: 8.82s
14:	learn: 0.0075782	total: 131ms	remaining: 8.6s
15:	learn: 0.0075396	total: 135ms	remaining: 8.32s
16:	learn: 0.0075064	total: 140ms	remaining: 8.09s
17:	learn: 0.0074781	total: 144ms	remaining: 7.88s
18:	learn: 0.0074534	total: 149ms	remaining: 7.7s
19:	learn

195:	learn: 0.0070148	total: 973ms	remaining: 3.99s
196:	learn: 0.0070139	total: 978ms	remaining: 3.98s
197:	learn: 0.0070125	total: 983ms	remaining: 3.98s
198:	learn: 0.0070117	total: 988ms	remaining: 3.98s
199:	learn: 0.0070107	total: 992ms	remaining: 3.97s
200:	learn: 0.0070096	total: 997ms	remaining: 3.96s
201:	learn: 0.0070086	total: 1s	remaining: 3.96s
202:	learn: 0.0070074	total: 1s	remaining: 3.95s
203:	learn: 0.0070064	total: 1.01s	remaining: 3.94s
204:	learn: 0.0070055	total: 1.01s	remaining: 3.94s
205:	learn: 0.0070045	total: 1.02s	remaining: 3.93s
206:	learn: 0.0070032	total: 1.02s	remaining: 3.92s
207:	learn: 0.0070023	total: 1.03s	remaining: 3.92s
208:	learn: 0.0070017	total: 1.03s	remaining: 3.91s
209:	learn: 0.0070010	total: 1.04s	remaining: 3.9s
210:	learn: 0.0069998	total: 1.04s	remaining: 3.9s
211:	learn: 0.0069986	total: 1.05s	remaining: 3.89s
212:	learn: 0.0069978	total: 1.05s	remaining: 3.88s
213:	learn: 0.0069969	total: 1.06s	remaining: 3.88s
214:	learn: 0.006996

391:	learn: 0.0068528	total: 1.95s	remaining: 3.02s
392:	learn: 0.0068521	total: 1.96s	remaining: 3.02s
393:	learn: 0.0068516	total: 1.96s	remaining: 3.02s
394:	learn: 0.0068509	total: 1.97s	remaining: 3.01s
395:	learn: 0.0068501	total: 1.97s	remaining: 3.01s
396:	learn: 0.0068493	total: 1.98s	remaining: 3s
397:	learn: 0.0068485	total: 1.98s	remaining: 3s
398:	learn: 0.0068479	total: 1.99s	remaining: 2.99s
399:	learn: 0.0068470	total: 1.99s	remaining: 2.99s
400:	learn: 0.0068460	total: 2s	remaining: 2.98s
401:	learn: 0.0068449	total: 2s	remaining: 2.98s
402:	learn: 0.0068441	total: 2.01s	remaining: 2.97s
403:	learn: 0.0068436	total: 2.01s	remaining: 2.97s
404:	learn: 0.0068430	total: 2.02s	remaining: 2.96s
405:	learn: 0.0068424	total: 2.02s	remaining: 2.96s
406:	learn: 0.0068418	total: 2.03s	remaining: 2.95s
407:	learn: 0.0068412	total: 2.03s	remaining: 2.95s
408:	learn: 0.0068405	total: 2.04s	remaining: 2.94s
409:	learn: 0.0068397	total: 2.04s	remaining: 2.94s
410:	learn: 0.0068388	to

555:	learn: 0.0067423	total: 2.73s	remaining: 2.18s
556:	learn: 0.0067414	total: 2.73s	remaining: 2.17s
557:	learn: 0.0067409	total: 2.74s	remaining: 2.17s
558:	learn: 0.0067403	total: 2.75s	remaining: 2.17s
559:	learn: 0.0067396	total: 2.75s	remaining: 2.16s
560:	learn: 0.0067390	total: 2.75s	remaining: 2.16s
561:	learn: 0.0067381	total: 2.76s	remaining: 2.15s
562:	learn: 0.0067374	total: 2.76s	remaining: 2.15s
563:	learn: 0.0067367	total: 2.77s	remaining: 2.14s
564:	learn: 0.0067363	total: 2.77s	remaining: 2.14s
565:	learn: 0.0067358	total: 2.78s	remaining: 2.13s
566:	learn: 0.0067350	total: 2.78s	remaining: 2.13s
567:	learn: 0.0067346	total: 2.79s	remaining: 2.12s
568:	learn: 0.0067337	total: 2.79s	remaining: 2.12s
569:	learn: 0.0067331	total: 2.8s	remaining: 2.11s
570:	learn: 0.0067325	total: 2.8s	remaining: 2.11s
571:	learn: 0.0067322	total: 2.81s	remaining: 2.1s
572:	learn: 0.0067316	total: 2.81s	remaining: 2.1s
573:	learn: 0.0067310	total: 2.82s	remaining: 2.09s
574:	learn: 0.00

716:	learn: 0.0066468	total: 3.51s	remaining: 1.39s
717:	learn: 0.0066462	total: 3.52s	remaining: 1.38s
718:	learn: 0.0066455	total: 3.52s	remaining: 1.38s
719:	learn: 0.0066451	total: 3.53s	remaining: 1.37s
720:	learn: 0.0066446	total: 3.53s	remaining: 1.37s
721:	learn: 0.0066442	total: 3.54s	remaining: 1.36s
722:	learn: 0.0066437	total: 3.54s	remaining: 1.36s
723:	learn: 0.0066432	total: 3.55s	remaining: 1.35s
724:	learn: 0.0066428	total: 3.56s	remaining: 1.35s
725:	learn: 0.0066422	total: 3.56s	remaining: 1.34s
726:	learn: 0.0066417	total: 3.57s	remaining: 1.34s
727:	learn: 0.0066412	total: 3.57s	remaining: 1.33s
728:	learn: 0.0066408	total: 3.58s	remaining: 1.33s
729:	learn: 0.0066403	total: 3.58s	remaining: 1.32s
730:	learn: 0.0066397	total: 3.59s	remaining: 1.32s
731:	learn: 0.0066391	total: 3.59s	remaining: 1.31s
732:	learn: 0.0066387	total: 3.6s	remaining: 1.31s
733:	learn: 0.0066383	total: 3.6s	remaining: 1.3s
734:	learn: 0.0066378	total: 3.61s	remaining: 1.3s
735:	learn: 0.00

875:	learn: 0.0065663	total: 4.29s	remaining: 608ms
876:	learn: 0.0065660	total: 4.3s	remaining: 603ms
877:	learn: 0.0065656	total: 4.3s	remaining: 598ms
878:	learn: 0.0065650	total: 4.31s	remaining: 593ms
879:	learn: 0.0065645	total: 4.32s	remaining: 589ms
880:	learn: 0.0065640	total: 4.32s	remaining: 584ms
881:	learn: 0.0065634	total: 4.33s	remaining: 579ms
882:	learn: 0.0065631	total: 4.33s	remaining: 574ms
883:	learn: 0.0065628	total: 4.34s	remaining: 569ms
884:	learn: 0.0065622	total: 4.34s	remaining: 564ms
885:	learn: 0.0065617	total: 4.35s	remaining: 560ms
886:	learn: 0.0065613	total: 4.35s	remaining: 555ms
887:	learn: 0.0065609	total: 4.36s	remaining: 550ms
888:	learn: 0.0065604	total: 4.36s	remaining: 545ms
889:	learn: 0.0065599	total: 4.37s	remaining: 540ms
890:	learn: 0.0065593	total: 4.37s	remaining: 535ms
891:	learn: 0.0065588	total: 4.38s	remaining: 530ms
892:	learn: 0.0065585	total: 4.38s	remaining: 525ms
893:	learn: 0.0065579	total: 4.39s	remaining: 520ms
894:	learn: 0.

In [15]:
#loop months
#for January: filter Jan
#predict Jan
#update all lags
#loop

predictor_table=zri3.copy()

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    X_test2=X_test.copy()
    
    #run prediction for one month
    X_test2=X_test2[X_test2['year-month']==month]
    
    X_test2['CountyNameEnc'] = labelencoder.transform(X_test2['CountyName'])
    
    X_test2=X_test2.drop(['CountyName','RegionName','City','State','Metro','year-month'],axis=1)
    val=cat.predict(X_test2)
    
    #define nextmonth
    nextmonth=date_mapping[[mon_ind for mon_ind, name in date_mapping.items() if name == month][0]+1]

    #write current month prediction into predictor_table
    if nextmonth<'2020-01':
        predictor_table.loc[predictor_table['year-month']==nextmonth,'ZRI']=val
        
        predictor_table=laggenerator(1, 'ZRI',predictor_table)
        predictor_table=laggenerator(2, 'ZRI',predictor_table)
        predictor_table=laggenerator(3, 'ZRI',predictor_table)
        predictor_table=laggenerator(4, 'ZRI',predictor_table)
        predictor_table=laggenerator(5, 'ZRI',predictor_table)
        predictor_table=laggenerator(6, 'ZRI',predictor_table)
        
        X_test=predictor_table[predictor_table['year-month']>'2018-12']



In [16]:
RSME=(sum((Y_test['ZRI']-X_test['ZRI'])**2)/len(Y_test))**0.5
RSME

0.01006410819766225

In [70]:
R2=1-sum((Y_test['ZRI']-X_test['ZRI'])**2)/sum((Y_test['ZRI']-Y_test.mean()[0])**2)
R2

0.06292780536214204

187597   -0.010590
189458   -0.002483
191319    0.005713
193180    0.005660
195041    0.000222
            ...   
200523    0.001937
202384    0.006423
204245   -0.007027
206106    0.003412
207967   -0.023399
Name: ZRI, Length: 15192, dtype: float64

In [63]:
Y_test.mean()[0]

0.0015732354761700603

In [17]:
merged=X_test.merge(Y_test, left_index=True, right_index=True)

In [18]:
import plotly.express as px

In [36]:
px.line(zri2.iloc[0:700], x='year-month', y='ZRI')

In [45]:
len(merged['RegionID'].drop_duplicates())

1266

In [40]:
merged

,RegionID,RegionName,City,State,Metro,CountyName,year-month,ZRI_x,ZRI_lag1,ZRI_lag2,ZRI_lag3,ZRI_lag4,ZRI_lag5,ZRI_lag6,ZRI_y
187597,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-01,-0.009017,-0.005381,-0.008007,-0.012302,-0.008711,-0.003472,-0.003460,-0.009017
189458,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-02,0.000834,-0.009017,-0.005381,-0.008007,-0.012302,-0.008711,-0.003472,-0.000910
191319,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-03,-0.002216,0.000834,-0.009017,-0.005381,-0.008007,-0.012302,-0.008711,0.007286
193180,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-04,0.005303,-0.002216,0.000834,-0.009017,-0.005381,-0.008007,-0.012302,0.007233
195041,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-05,0.003683,0.005303,-0.002216,0.000834,-0.009017,-0.005381,-0.008007,0.001795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200523,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-08,0.007367,-0.002414,0.008446,-0.004490,0.004763,-0.011502,0.004144,0.003511
202384,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-09,-0.001426,0.007367,-0.002414,0.008446,-0.004490,0.004763,-0.011502,0.007996
204245,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-10,0.005973,-0.001426,0.007367,-0.002414,0.008446,-0.004490,0.004763,-0.005454
206106,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-11,-0.000652,0.005973,-0.001426,0.007367,-0.002414,0.008446,-0.004490,0.004985


In [21]:
#enc = OneHotEncoder(handle_unknown='ignore')

#X_train_enc = pd.DataFrame(enc.fit_transform(X_train[['CountyName']]).toarray())

#X_train = X_train.join(X_train_enc)
#X_train

#,

In [22]:
testzip=zri3[zri3['RegionID']==753844]
testzip['year-month']=pd.to_datetime(testzip['year-month'])
testzip=testzip.set_index('year-month')
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

decomposed=seasonal_decompose(testzip['ZRI'],model='additive')

trend=decomposed.trend
seasonal=decomposed.seasonal
residual=decomposed.resid

#px.line(testzip['ZRI'])
#px.line(trend)
#px.line(seasonal)
#px.line(residual)

ValueError: You must specify a period or x must be a pandas object with a DatetimeIndex with a freq not set to None

In [ ]:
from pmdarima.arima import auto_arima

x=100
X_train, X_test=testzip[0:x], testzip[x:len(testzip)-1]

arima_model=auto_arima(X_train['ZRI'], start_p=1, start_q=1, max_p=5, max_q=5, m=12, start_P=0, start_D=0,
                       start_Q=0, max_P=5, max_D=5, max_Q=5, 
                       seasonal=True, trace=True, stepwise=True, n_fits=50)

In [ ]:
zhi_reshaped2.gapsize==0

In [ ]:
zhi_reshaped2[(zhi_reshaped2['gapsize']==0)]

In [ ]:
zhi_reshaped2.fillna(0)
zhi_reshaped2['gapsize']=zhi_reshaped2['Value'].rolling(1).sum()

In [ ]:
zhi_reshaped2['gapsize']

In [ ]:
zhi_df.isna().sum(axis=1).shape

In [ ]:
zhi_df.isna().sum(axis=1)<1

In [ ]:
zhi_df[(zhi_df.isna().sum(axis=1)>8)].shape

In [ ]:
zhi_reshaped.groupby('RegionID').agg('count')

In [ ]:
zhi_reshaped

In [ ]:
acs_df=pd.read_csv('../data/acs_5yr_merged.csv')

In [ ]:
acs_df.shape

In [ ]:
acs_df.isna().sum()